In [1]:
#    This file is part of DEAP.
#
#    DEAP is free software: you can redistribute it and/or modify
#    it under the terms of the GNU Lesser General Public License as
#    published by the Free Software Foundation, either version 3 of
#    the License, or (at your option) any later version.
#
#    DEAP is distributed in the hope that it will be useful,
#    but WITHOUT ANY WARRANTY; without even the implied warranty of
#    MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the
#    GNU Lesser General Public License for more details.
#
#    You should have received a copy of the GNU Lesser General Public
#    License along with DEAP. If not, see <http://www.gnu.org/licenses/>.

import array
import random
import math

import matplotlib.pyplot as plt
import copy

import numpy as np
from scipy.integrate import odeint

import time


import numpy

from deap import algorithms
from deap import base
from deap import benchmarks
from deap import creator
from deap import tools


arquivo = open("GRN5.txt", 'r')

x = []
A = []
B = []
C = []
D = []
E = []

for linha in arquivo:
    elementos = linha.split()
    x.append(float(elementos[0].strip()))
    A.append(float(elementos[1].strip()))
    B.append(float(elementos[2].strip()))
    C.append(float(elementos[3].strip()))
    D.append(float(elementos[4].strip()))
    E.append(float(elementos[5].strip()))


maximo_A = max(A)
maximo_B = max(B)
maximo_C = max(C)
maximo_D = max(D)
maximo_E = max(E)


maximos = []
maximos.append(maximo_A)
maximos.append(maximo_B)
maximos.append(maximo_C)
maximos.append(maximo_D)
maximos.append(maximo_E)


A_ORIGINAL = copy.deepcopy(A)
B_ORIGINAL = copy.deepcopy(B)
C_ORIGINAL = copy.deepcopy(C)
D_ORIGINAL = copy.deepcopy(D)
E_ORIGINAL = copy.deepcopy(E)

maior_todos = max(maximos)


Y0 = []
Y0.append(A[0])
Y0.append(B[0])
Y0.append(C[0])
Y0.append(D[0])
Y0.append(E[0])

dobra_pontos = copy.deepcopy(x)

IND_SIZE = 15

MIN_STRATEGY = 0.1
MAX_STRATEGY = 10

MIN_K = 0.1#Menor valor que K pode assumir
MAX_K = 1#Maior valor que K pode assumir
MIN_N = 1#Menor valor que N pode assumir
MAX_N = 25#Maior valor que N pode assumir
MIN_TAU = 0.1#Menor valor que TAU pode assumir
MAX_TAU = 5#Maior valor que TAU pode assumir

TAU_SIZE = 5
N_SIZE = 5
K_SIZE = 5

creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", array.array, typecode="d", fitness=creator.FitnessMin, strategy=None)
creator.create("Strategy", array.array, typecode="d")

def twoBody(y, t, tauA, kA, nA, tauB, kB, nB, tauC, kC, nC, tauD, kD, nD, tauE, kE, nE):
    ydot = np.empty((5,))
    
    ydot[0] = ((1 - (pow((y[4]/maximo_E),nA)/(pow((y[4]/maximo_E),nA)+pow(kA, nA))))-(y[0]/maximo_A)) / tauA

    ydot[1] = (((pow((y[0]/maximo_A), nB))/(pow((y[0]/maximo_A),nB) + pow(kB, nB))) - (y[1]/maximo_B)) / tauB

    ydot[2] = (((pow((y[1]/maximo_B), nC))/(pow((y[1]/maximo_B),nC) + pow(kC, nC))) - (y[2]/maximo_C)) / tauC

    ydot[3] = (((pow((y[2]/maximo_C), nD))/(pow((y[2]/maximo_C),nD) + pow(kD, nD))) - (y[3]/maximo_D)) / tauD

    ydot[4] = (((pow((y[3]/maximo_D), nE))/(pow((y[3]/maximo_D),nE) + pow(kE, nE))) - (y[4]/maximo_E)) / tauE

    return ydot

# Individual generator
def generateES(icls, scls, indsize, tausize, mintau, maxtau, ksize, mink, maxk, nsize, minn, maxn, smin, smax):
    #ind = icls(random.uniform(imin, imax) for _ in range(size))
    ind_local = []
    for _ in range(tausize):
        ind_local.append(random.uniform(mintau, maxtau))
    for _ in range(ksize):
        ind_local.append(random.uniform(mink, maxk))
    for _ in range(nsize):
        ind_local.append(random.uniform(minn, maxn))

        
    ind = icls(ind_local)  
    ind.strategy = scls(random.uniform(smin, smax) for _ in range(indsize))
    
    return ind

def checkStrategy(minstrategy):
    def decorator(func):
        def wrappper(*args, **kargs):
            children = func(*args, **kargs)
            for child in children:
                for i, s in enumerate(child.strategy):
                    if s < minstrategy:
                        child.strategy[i] = minstrategy
            return children
        return wrappper
    return decorator

def checkParam(mintau, maxtau, mink, maxk, minn, maxn):
    def decorator(func):
        def wrappper(*args, **kargs):
            children = func(*args, **kargs)
            for child in children:
                for i, s in enumerate(child):
                    #print("OLHA")
                    #print(i)
                    if i < 5:
                        if child[i] < mintau:
                            child[i] = mintau
                        if child[i] > maxtau:
                            child[i] = maxtau
                    if i >= 5 and i < 10:
                        if child[i] < mink:
                            child[i] = mink
                        if child[i] > maxk:
                            child[i] = maxk
                    if i >= 10:
                        if child[i] < minn:
                            child[i] = minn
                        if child[i] > maxn:
                            child[i] = maxn                        
            return children
        return wrappper
    return decorator

def organiza_pontos(solucao):
    pA = []
    pB = []
    pC = []
    pD = []
    pE = []
    for pontos in range(len(solucao)):
        if pontos % 2 == 0 or pontos % 2 == 1:
            pA.append(solucao[pontos][0])
            pB.append(solucao[pontos][1])
            pC.append(solucao[pontos][2])
            pD.append(solucao[pontos][3])
            pE.append(solucao[pontos][4])
    return pA, pB, pC, pD, pE


def calcula_diferenca(pA, pB, pC, pD, pE):
    difA = 0
    difB = 0
    difC = 0
    difD = 0
    difE = 0
    pAl = []
    pBl = []
    pCl = []
    pDl = []
    pEl = []
    #for elemento in range(len(pA)):
    #    pAl.append(pA[elemento] * maximo_A)
    #for elemento in range(len(pB)):
    #    pBl.append(pB[elemento] * maximo_B)
    #for elemento in range(len(pC)):
    #    pCl.append(pC[elemento] * maximo_C)
    #for elemento in range(len(pD)):
    #    pDl.append(pD[elemento] * maximo_D)
    #for elemento in range(len(pE)):
    #    pEl.append(pE[elemento] * maximo_E)

        
    for elemento in range(len(pA)):
        dif = abs(A_ORIGINAL[elemento] - pA[elemento])
        difA += dif
    for elemento in range(len(pB)):
        dif = abs(B_ORIGINAL[elemento] - pB[elemento])
        difB += dif
    for elemento in range(len(pC)):
        dif = abs(C_ORIGINAL[elemento] - pC[elemento])
        difC += dif
    for elemento in range(len(pD)):
        dif = abs(D_ORIGINAL[elemento] - pD[elemento])
        difD += dif
    for elemento in range(len(pE)):
        dif = abs(E_ORIGINAL[elemento] - pE[elemento])
        difE += dif
    
    difTotal = difA + difB + difC + difD + difE
    return difTotal


def calculaAPT(individual):
    #print("entrou")
    ind_atual = []
    for elemento in individual:
        ind_atual.append(elemento)


    tauA = ind_atual[0]
    tauB = ind_atual[1]
    tauC = ind_atual[2]
    tauD = ind_atual[3]
    tauE = ind_atual[4]
    kA = ind_atual[5]
    kB = ind_atual[6]
    kC = ind_atual[7]
    kD = ind_atual[8]
    kE = ind_atual[9]
    nA = ind_atual[10]
    nB = ind_atual[11]
    nC = ind_atual[12]
    nD = ind_atual[13]
    nE = ind_atual[14]
    solution = odeint(twoBody, Y0, dobra_pontos, args=(tauA, kA, int(nA), tauB, kB, int(nB), tauC, kC, int(nC), tauD, kD, int(nD), tauE, kE, int(nE),))
    pA, pB, pC, pD, pE = organiza_pontos(solution)    

    return calcula_diferenca(pA, pB, pC, pD, pE),

toolbox = base.Toolbox()
toolbox.register("individual", generateES, creator.Individual, creator.Strategy,
    IND_SIZE, TAU_SIZE, MIN_TAU, MAX_TAU, K_SIZE, MIN_K, MAX_K, N_SIZE, MIN_N, MAX_N, MIN_STRATEGY, MAX_STRATEGY)

toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("mate", tools.cxESBlend, alpha=0.1)
toolbox.register("mutate", tools.mutESLogNormal, c=1.0, indpb=0.03)
toolbox.register("select", tools.selTournament, tournsize=3)
#toolbox.register("evaluate", benchmarks.sphere)
toolbox.register("evaluate", calculaAPT)

toolbox.decorate("mate", checkStrategy(MIN_STRATEGY))
toolbox.decorate("mutate", checkStrategy(MIN_STRATEGY))
toolbox.decorate("mutate", checkParam(MIN_TAU, MAX_TAU, MIN_K, MAX_K, MIN_N, MAX_N))


def plotagrafico(individual):
    

    ind_atual = individual[0].tolist()
    tauA = ind_atual[0]
    tauB = ind_atual[1]
    tauC = ind_atual[2]
    tauD = ind_atual[3]
    tauE = ind_atual[4]
    kA = ind_atual[5]
    kB = ind_atual[6]
    kC = ind_atual[7]
    kD = ind_atual[8]
    kE = ind_atual[9]
    nA = ind_atual[10]
    nB = ind_atual[11]
    nC = ind_atual[12]
    nD = ind_atual[13]
    nE = ind_atual[14]

    solution = odeint(twoBody, Y0, dobra_pontos, args=(tauA, kA, int(nA), tauB, kB, int(nB), tauC, kC, int(nC), tauD, kD, int(nD), tauE, kE, int(nE),))
    pA, pB, pC, pD, pE = organiza_pontos(solution)
    
    plt.plot(x, pA)
    plt.plot(x, A_ORIGINAL)
    plt.legend(['Ac', 'A'])
    #plt.savefig("A.eps", dpi=300)
    plt.show()
    plt.plot(x, pB)
    plt.plot(x, B_ORIGINAL)
    plt.legend(['Bc', 'B'])
    #plt.savefig("B.eps", dpi=300)
    plt.show()
    plt.plot(x, pC)
    plt.plot(x, C_ORIGINAL)
    plt.legend(['Cc', 'C'])
    #plt.savefig("C.eps", dpi=300)
    plt.show()
    plt.plot(x, pD)
    plt.plot(x, D_ORIGINAL)
    plt.legend(['Dc', 'D'])
    #plt.savefig("D.eps", dpi=300)
    plt.show()
    plt.plot(x, pE)
    plt.plot(x, E_ORIGINAL)
    plt.legend(['Ec', 'E'])
    #plt.savefig("E.eps", dpi=300)
    plt.show()
        
def main():
    random.seed(1)
    MU, LAMBDA = 15, 105
    pop = toolbox.population(n=MU)
    hof = tools.HallOfFame(1)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", numpy.mean)
    stats.register("std", numpy.std)
    stats.register("min", numpy.min)
    stats.register("max", numpy.max)
    
    pop, logbook = algorithms.eaMuPlusLambda(pop, toolbox, mu=MU, lambda_=LAMBDA, 
        cxpb=0.1, mutpb=0.8, ngen=1000, stats=stats, halloffame=hof)
    print(hof)
    plotagrafico(hof)
    return pop, logbook, hof
    
if __name__ == "__main__":
    main()


gen	nevals	avg    	std    	min    	max    
0  	15    	150.574	18.6291	117.442	180.777
1  	88    	134.19 	12.8072	117.442	153.005
2  	98    	126.429	11.4057	115.769	149.687
3  	96    	118.043	2.45619	115.769	124.859
4  	100   	115.885	2.13768	109.499	119.525
5  	87    	113.768	4.1001 	99.845 	117.442
6  	96    	110.952	5.78999	99.845 	115.769
7  	95    	106.689	10.8396	73.3488	115.31 
8  	94    	98.448 	13.6151	73.3488	111.635
9  	99    	94.4571	11.4207	73.3488	111.715
10 	95    	85.1981	9.98006	62.528 	99.845 
11 	89    	74.6306	8.63651	62.528 	95.8585
12 	92    	70.8488	9.9044 	62.528 	95.8585


C:\Users\anabi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\scipy\integrate\_odepack_py.py:248: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)


13 	95    	66.3384	7.70126	61.0991	88.0467
14 	95    	62.5137	1.27832	60.9389	66.7596
15 	92    	60.9621	1.2782 	58.433 	62.528 
16 	95    	60.1341	1.45631	57.2955	62.528 
17 	87    	59.292 	1.08982	57.2955	60.9389


KeyboardInterrupt: 